# Variáveis aleatórias:

$X_1 = R = \text{Lognormal com }\mu_R = 249,89 \text{kN e  }\sigma_R = 24.978 \text{kN}$

$X_2 = S_{pp} = \text{Normal com }\mu_{pp} = 19.86 \text{kN e  }\sigma_{pp} = 0.993 \text{kN}$

$X_3 = S_{sv} = \text{Gumbel Tipo I (máximo) com }\mu_{sv} = 78.84 \text{kN e  }\sigma_{pp} = 15.768 \text{kN}$

# Equação de Estado Limite

$g(\underline{X}) = X_1 - X_2 - X_3$

# Dados de entrada

In [1]:
# Vetor de médias:
M = [249.78, 19.86, 78.84]

# Vetor de desvios padrão:
D = [24.978, 0.993, 15.768]

# Precisão fixada:
delta = 0.00001

# Algoritmo

## Carrega as bibliotecas necessárias

In [2]:
# import autograd.numpy as np
import numpy as np
from autograd import grad
from scipy.stats import lognorm, norm, gumbel_r
from copy import copy

## Define as funções que serão utilizadas

In [3]:
# Função do estado limite
def calcular_gx(X):
    return X[0] - X[1] - X[2]

# Função para obter o gradiente
def __calcular_gradiente(X):
    return grad(calcular_gx)(X)

def calcular_gradiente_z(X, J):
    grad_x = __calcular_gradiente(X)
    return np.matmul(np.linalg.inv(J), grad_x)

def calcular_modulo(vetor):
    return np.sqrt(sum([x**2 for x in vetor]))

# Função para calcular o coeficiente de variação:
def __calcular_cv(media, dpadrao):
    return dpadrao / media

# Função para calcular os parâmetros da Lognormal:
def calcular_parametros_lognormal(media, dpadrao):
    cv = __calcular_cv(media, dpadrao)
    params = {}
    params['epsilon'] = np.sqrt(np.log(1 + cv**2))
    params['lambda'] = np.log(media) - 1/2 * params['epsilon']**2
    return params

# Função para calcular os parâmetros da Gumbel:
def calcular_parametros_gumbel(media, dpadrao):
    cv = __calcular_cv(media, dpadrao)
    params = {}
    params['alpha'] = np.pi / np.sqrt(6 * dpadrao**2)
    params['u'] = media  - (0.577216 / params['alpha'])
    return params

# Função para calcular os parâmetros da Normal equivalente:
def calcular_parametros_normal_equiv(acumulada, densidade, media):
    inversa = norm.ppf(acumulada)
    phi = norm.pdf(inversa)
    params = {}
    params['sigma'] = phi / densidade
    params['mi'] = media - params['sigma'] * inversa
    return params

## Obtem os parâmetros da Lognormal e Gumbel

In [4]:
# Parâmetros da Lognormal
params_x1 = calcular_parametros_lognormal(M[0], D[0])

# Parâmetros da distribuição de Gumbel
params_x3 = calcular_parametros_gumbel(M[2], D[2])

## Transforma os dados de entrada para o formato adequado

In [5]:
# Matriz de desvios padrão

D = np.array(D, dtype=float)
D = np.diag(D)

## Inicia o processo iterativo

### Iteração k = 0

In [6]:
# Iteração:
k = 0

# Ponto do projeto no espaço original
X = copy(M)

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

beta = []
L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Calculo da estimativa no novo ponto
modulo_gradz = calcular_modulo(grad_z)
Z_ = (1 / modulo_gradz**2) *(np.matmul(grad_z, Z) - calcular_gx(X)) * grad_z

### Iteração k = 1

In [7]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
(beta[k] - beta[k-1]) / beta[k] <= delta

False

### Iteração k = 2

In [8]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
abs(beta[k] - beta[k-1]) / beta[k] <= delta

False

### Iteração k = 3

In [9]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
abs(beta[k] - beta[k-1]) / beta[k] <= delta

False

### Iteração k = 4

In [11]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
abs(beta[k] - beta[k-1]) / beta[k] <= delta

False

### Iteração k = 5

In [13]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
abs(beta[k] - beta[k-1]) / beta[k] <= delta

False

### Iteração k = 6

In [15]:
k += 1

# Ponto do projeto no espaço original
X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))

# Parâmetros da Normal Equivalente à Lognormal
acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])

# Parâmetros da Normal Equivalente à Gumbel
acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])

L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float)
M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']]

# Matriz Jacobiana
J = np.matmul(L, np.linalg.inv(D))

# Ponto do projeto no espaço reduzido:
Z = np.matmul(J, np.subtract(X, M_))

# Calcula o gradiente:
grad_z = calcular_gradiente_z(X, J)

# Cálculo da estimativa do índice de confiabilidade:
beta.append(calcular_modulo(Z))

# Verificação da tolerância
abs(beta[k] - beta[k-1]) / beta[k] <= delta

False

In [16]:
beta

[0.17670487238978136,
 5.063590436753624,
 4.127072480026653,
 4.841783178130328,
 4.380389951985237,
 4.41359674992093,
 3.652348202163642]